In [79]:
import matplotlib.pyplot as plt
import numpy as np
import os
# import PIL
import tensorflow as tf
import pickle as pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import glob
from random import shuffle
import datetime

In [80]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [81]:
# import pathlib
# data_dir = pathlib.Path('E:/Users/sentinel_industry/downloaded_aois/')


In [82]:
x_size = 230
y_size = 230
def load_features(name):
    decoded = name.decode("UTF-8")
    if os.path.exists(decoded):
        with open(decoded, 'rb') as f:
            file = pickle.load(f)
            label = tf.strings.split(tf.strings.split(name, '/')[-1], '\\')[-2]
            if label == 'coal':
                label = [1,0,0]
            elif label == 'steel':
                label = [0,1,0]
            else: label = [0,0,1]
            if (file["B02"].shape[1]<230 or file["B02"].shape[2]<230): 
                print("oh oh, downloaded patch too smol:", decoded)
            B02, B03, B04, B08 = file['B02'][0][0:x_size,0:y_size], file['B03'][0][0:x_size,0:y_size], file['B04'][0][0:x_size,0:y_size],file['B08'][0][0:x_size,0:y_size]
            for channel in [ B02, B03, B04, B08]: channel = channel/channel.max()
            features = np.array([ B02, B03, B04, B08]).transpose(1,2,0)
            # features = np.expand_dims(features, axis=0)
            return features, label
            # I have commented the line below but this should return
            # the features and the label in a one hot vector
            # return file['features'], file['targets']
    else:
        print("Something went wrong!")
        exit(-1)

In [83]:
path = 'E:/Users/sentinel_industry/downloaded_aois/coal/'

pkl_files = glob.glob((path+"*.pickle"))


In [84]:
print(len(pkl_files))

2258


In [85]:
path_steel = 'E:/Users/sentinel_industry/downloaded_aois/steel/'
pkl_files_steel = glob.glob((path_steel+"*.pickle"))

In [86]:
path_steel_no_ind = 'E:/Users/sentinel_industry/downloaded_aois/steel/no_ind/'
pkl_files_steel_no_ind = glob.glob((path_steel_no_ind+"*.pickle"))

In [87]:
path_coal_no_ind = 'E:/Users/sentinel_industry/downloaded_aois/coal/no_ind/'
pkl_files_coal_no_ind = glob.glob((path_coal_no_ind+"*.pickle"))

In [88]:
print(len(pkl_files_steel))

742


In [89]:
pkl_files = pkl_files + pkl_files_steel + pkl_files_steel_no_ind + pkl_files_coal_no_ind
shuffle(pkl_files)

In [90]:
# from itertools import chain

# pkl_files = list(chain.from_iterable(zip(pkl_files, pkl_files_steel)))


In [91]:
print(len(pkl_files))

7097


In [92]:
dataset_len=len(pkl_files)

In [93]:
dataset = tf.data.Dataset.from_tensor_slices((pkl_files))

def data_loader(filename):
    features, labels = tf.numpy_function(load_features, [filename], [tf.uint16, tf.int32])
    # features.set_shape((None, 242,242,4))
    # labels.set_shape(( 1))
    return features, labels

dataset = dataset.map(data_loader)
               # load_features, [filename], [tf.uint16, tf.string])))

In [94]:
dataset.shuffle(100, seed=42)

train_size = np.floor(0.8 * dataset_len)
valid_size = np.floor(0.1 * dataset_len)
test_size = np.floor(0.1 * dataset_len)

train = dataset.take(train_size)
remaining = dataset.skip(train_size)
valid = remaining.take(valid_size)
test = remaining.skip(valid_size)

In [95]:
train = train.shuffle(100,  reshuffle_each_iteration=True)

In [96]:
in_seed = 42
resize_rescale_augment = tf.keras.Sequential([
    layers.RandomRotation(0.2, seed=in_seed, input_shape=(x_size,y_size,4)),  # here, maybe try larger values
    # layers.RandomCrop(120,120, seed=in_seed, input_shape=(x_size,y_size,4)),  # to be changed with centercrop for data augmentation/ use centre first, then random?
    layers.CenterCrop(120,120, ),
    # layers.Rescaling(1./10000),#,input_shape=(240,240,4)), # I think the data is scaled to 10000
    layers.RandomFlip("horizontal_and_vertical", seed=in_seed), 
])

In [97]:
# dataset = dataset.map((
#   lambda x, y: (resize_and_rescale(x), y)))

In [98]:
for image_batch, labels_batch in dataset:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(230, 230, 4)
(3,)


In [99]:
train

<ShuffleDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.uint16, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

In [100]:
pwd

'E:\\Users\\sentinel_industry'

In [101]:
model = tf.keras.Sequential([
    resize_rescale_augment,
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3)
])
# model = tf.keras.Sequential([
#     resize_rescale_augment,
#     layers.Conv2D(3, 5, padding='same', activation='relu'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
#                                                             # maybe try changing the source so it accepts 4 channels?
#     tf.keras.applications.resnet50.ResNet50(
#         include_top=False,
#         weights=None, # if I don't use the pre trained weights from image net, does it matter that i don't use the preprocessing step which reorders RGB to BGR and zero-centers wrt imagenet?
#         input_shape=(120, 120, 3),
#         pooling=max ,
#         classes=3,),
#     layers.Flatten(), # does this make sense? or is there another way to get down to just three output dimensions?
#     layers.Dense(3)
# ])

log_dir = "E:/Users/sentinel_industry/logs/train/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.BinaryAccuracy()])


In [102]:
model.build()

In [103]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_6 (Sequential)   (None, 120, 120, 4)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 120, 120, 16)      592       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 60, 60, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 60, 60, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 30, 30, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 30, 30, 64)       

In [104]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [105]:
%tensorboard --logdir='E:/Users/sentinel_industry/logs/train/'

Reusing TensorBoard on port 6006 (pid 17616), started 0:14:23 ago. (Use '!kill 17616' to kill it.)

In [106]:
epochs=60
batch_size = 40
history = model.fit(
    train.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE),
    validation_data=valid.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE),
    validation_steps=1,
    epochs=epochs,
    callbacks=[tensorboard_callback]
)

Epoch 1/60
141/141 [==============================] - 103s 709ms/step - loss: 27.1761 - binary_accuracy: 0.6606 - val_loss: 1.1262 - val_binary_accuracy: 0.6833
Epoch 2/60
141/141 [==============================] - 97s 685ms/step - loss: 0.9190 - binary_accuracy: 0.7623 - val_loss: 1.0327 - val_binary_accuracy: 0.6917
Epoch 3/60
141/141 [==============================] - 97s 683ms/step - loss: 0.7582 - binary_accuracy: 0.7783 - val_loss: 1.0571 - val_binary_accuracy: 0.7000
Epoch 4/60
 56/141 [==========>...................] - ETA: 33s - loss: 0.7396 - binary_accuracy: 0.7917

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir E:/Users/sentinel_industry/logs/train/

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(18, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
# plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
iterator=tf.compat.v1.data.make_one_shot_iterator(dataset)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    next_element = iterator.get_next()
    im = resize_rescale_augment(next_element[0])
    plt.imshow(np.flip(im[0,:,:,0:3], axis=2))
    plt.title(convert_onehot(next_element))

In [ ]:
def convert_onehot(next_element):
    if (next_element[1].numpy() == np.array([0, 0, 1])).all(): return 'no industry'
    elif (next_element[1].numpy() == np.array([0, 1, 0])).all(): return 'steel'
    elif (next_element[1].numpy() == np.array([1, 0, 0])).all(): return 'coal'
    else: return 'not valid'

In [ ]:
next_element[1].numpy().all()

In [ ]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("module.name", "/path/to/file.py")
foo = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = foo
spec.loader.exec_module(foo)

In [ ]:
! PATH